In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score # Confusion matrix 수업 때 진행할 예정
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

### 교차검증
- train/test로 나누게 되면 과적합이 발생할 확률이 높다(overfitting)
- 과적합은? 모델이 학습 데이터에 과도하게 최적화된다. -> 실제 다른 데이터로 수행하는 경우에는 예측 성능이 과도하게 떨어질 수 있다.
- train의 정확도가 높고 test는 낮다.
- train은 낮은데 test는 높다? # 문제가 있구나 생각할 것. 과대적합, 과소적합
- 이런 부분을 해결하기 위해서 교차검증을 진행한다.
- 모의고사 문제를 볼 때 랜덤하게 문제를 풀어본다.
- 1~100번 푸는 것보다 1번 풀고 50번 풀고 20번 풀고 이런 랜덤한 배치를 하게 되면 과적합을 피할 수 있다.
- 교차검증은 데이터의 편중을 막기위해 여러 개의 별도의 세트로 구성된 학습 데이터와 검증 데이터 세트로 학습과 평가를 수행하는 것
- 그 수행결과에 따라 하이퍼파라미터를 튜닝하는 등 모델 최적화를 진행하게 된다.

In [2]:
fold_iris = load_iris()
features = fold_iris.data

label = fold_iris.target
fold_df_clf = DecisionTreeClassifier()

### 교차검증 코드

In [3]:
kfold = KFold(n_splits=5) # 내가 n번만큼 폴드 세트별로 정확도를 볼 것인가?, 해당 정확도를 리스트에 담고 평균을 보거나 한다.
# 정확도
# 정밀도
# 재현율
cv_accuracy = []
cv_precision = []
cv_recall = []

In [4]:
kfold.split(features)

<generator object _BaseKFold.split at 0x000001CF803E0F20>

In [9]:
n_iter = 0 # 5번 진행 예정이니 초기값 0 먼저 지정
for train_idx, test_idx in kfold.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    # 학습을 진행
    fold_df_clf.fit(X_train, y_train)
    
    #예측
    fold_pred = fold_df_clf.predict(X_test)
    
    # 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 3)
    precision = np.round(precision_score(y_test, fold_pred), 3)
    recall = np.round(recall_score(y_test, fold_pred), 3)
    print("\n{} 교차검증 정확도 :{}, 교차검증 정밀도 :{} 교차검증 재현율 :{}".format(n_iter, accuracy, precision, recall))
    
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print("\n")
print("\n 평균 검증 정확도", np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 :1.0, 교차검증 정밀도 :0.0 교차검증 재현율 :0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [10]:
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

- iris 데이터는 타겟이 이진이 아니다.
- 0, 1, 2 세 가지로 나눠진다.
- recall, precision 얘는 타겟값이 이진이 아니면 계산할 때 문제가 생길 수 있다.

In [11]:
# 에러가 나지 않도록 하기 위해서는 precision과 recall값을 제외해야함
n_iter = 0 # 5번 진행 예정이니 초기값 0 먼저 지정
for train_idx, test_idx in kfold.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    # 학습을 진행
    fold_df_clf.fit(X_train, y_train)
    
    #예측
    fold_pred = fold_df_clf.predict(X_test)
    
    # 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 3)
    #precision = np.round(precision_score(y_test, fold_pred), 3)
    #recall = np.round(recall_score(y_test, fold_pred), 3)
    print("\n{} 교차검증 정확도 :{}, 교차검증 정밀도 :{} 교차검증 재현율 :{}".format(n_iter, accuracy, precision, recall))
    
    cv_accuracy.append(accuracy)
    #cv_precision.append(precision)
    #cv_recall.append(recall)
    
print("\n")
print("\n 평균 검증 정확도", np.mean(cv_accuracy))
#print("\n 평균 검증 정확도", np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 :1.0, 교차검증 정밀도 :0.0 교차검증 재현율 :0.0

2 교차검증 정확도 :1.0, 교차검증 정밀도 :0.0 교차검증 재현율 :0.0

3 교차검증 정확도 :0.9, 교차검증 정밀도 :0.0 교차검증 재현율 :0.0

4 교차검증 정확도 :0.933, 교차검증 정밀도 :0.0 교차검증 재현율 :0.0

5 교차검증 정확도 :0.8, 교차검증 정밀도 :0.0 교차검증 재현율 :0.0



 평균 검증 정확도 0.9139999999999999


- kfold의 문제점
- 정답비중에 대해서 고르게 나눠지는 게 아니라 랜덤하게 하다보니깐 정답이 0, 1, 2가 있는데, kfold 나눴을 때 0, 1만 포함되거나, 0만 포함되거나, 2만 포함되거나 1, 2만 포함되거나 이런 식으로 데이터가 나눠질 수 있다.

In [12]:
kfold_iris_data = load_iris()
kfold_iris_data_df = pd.DataFrame(data=kfold_iris_data.data, columns=kfold_iris_data.feature_names)
kfold_iris_data_df["target"] = kfold_iris_data.target
print(kfold_iris_data_df["target"].value_counts())

0    50
1    50
2    50
Name: target, dtype: int64


In [13]:
# kfold의 문제점을 보면
kfold_iris = KFold(n_splits=3)
kfold_iris.split(kfold_iris_data_df)

<generator object _BaseKFold.split at 0x000001CF81473350>

In [14]:
kfold_iris_data_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [15]:
cnt_iter = 0
for train_idx, test_idx in kfold_iris.split(kfold_iris_data_df):
    cnt_iter += 1
    label_train = kfold_iris_data_df["target"].iloc[train_idx]
    label_test = kfold_iris_data_df["target"].iloc[test_idx]
    print("교차검증 :{}".format(cnt_iter))
    print("학습 레이블 데이터 분포\n", label_train.value_counts())
    print("검증 레이블 데이터 분포\n", label_test.value_counts())

교차검증 :1
학습 레이블 데이터 분포
 1    50
2    50
Name: target, dtype: int64
검증 레이블 데이터 분포
 0    50
Name: target, dtype: int64
교차검증 :2
학습 레이블 데이터 분포
 0    50
2    50
Name: target, dtype: int64
검증 레이블 데이터 분포
 1    50
Name: target, dtype: int64
교차검증 :3
학습 레이블 데이터 분포
 0    50
1    50
Name: target, dtype: int64
검증 레이블 데이터 분포
 2    50
Name: target, dtype: int64


- kfold를 사용할 때 이진의 경우도 그렇지만, 이진 외 다중 분류 같은 경우는
- target의 값을 동일하게 가져가서 kfold 데이터가 한 쪽으로 몰리지 않게 할 수 있다.

### StratifiedKfold
- 분류 문제에서만 정답에 대한 이슈가 발생할 것. 0, 1 또는 0, 1, 2 분류에 대한 클래스의 불균형은 대부분 범주형 타겟에 나올 것이다.
- 회귀 문제와 같은 연속형 변수는 의미가 없다. 회귀 관련 예측할 때는 stratifiedKfold를 쓸 필요가 없다.

In [16]:
from sklearn.model_selection import StratifiedKFold

In [17]:
skf_iris = StratifiedKFold(n_splits=3)

In [18]:
cnt_iter = 0
for train_idx, test_idx in skf_iris.split(kfold_iris_data_df, kfold_iris_data_df["target"]): # 기존 데이터와 Y값도 같이 넣어야함
    cnt_iter += 1
    label_train = kfold_iris_data_df["target"].iloc[train_idx]
    label_test = kfold_iris_data_df["target"].iloc[test_idx]
    print("교차검증 :{}".format(cnt_iter))
    print("학습 레이블 데이터 분포\n", label_train.value_counts())
    print("검증 레이블 데이터 분포\n", label_test.value_counts())

교차검증 :1
학습 레이블 데이터 분포
 2    34
0    33
1    33
Name: target, dtype: int64
검증 레이블 데이터 분포
 0    17
1    17
2    16
Name: target, dtype: int64
교차검증 :2
학습 레이블 데이터 분포
 1    34
0    33
2    33
Name: target, dtype: int64
검증 레이블 데이터 분포
 0    17
2    17
1    16
Name: target, dtype: int64
교차검증 :3
학습 레이블 데이터 분포
 0    34
1    33
2    33
Name: target, dtype: int64
검증 레이블 데이터 분포
 1    17
2    17
0    16
Name: target, dtype: int64


In [19]:
result_skfold = StratifiedKFold(n_splits=3)
result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0 # 5번 진행 예정이니 초기값 0 먼저 지정

cv_accuracy = []
cv_precision = []
cv_recall = []

for train_idx, test_idx in result_skfold.split(features, label):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습을 진행
    result_clf.fit(X_train, y_train)
    
    #예측
    fold_pred = result_clf.predict(X_test)
    
    #정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, fold_pred), 3)
    precision = np.round(precision_score(y_test, fold_pred, average="macro"), 3)
    recall = np.round(recall_score(y_test, fold_pred, average="macro"), 3)
    print("\n{} 교차검증 정확도 :{}, 교차검증 정밀도 :{} 교차검증 재현율 :{}".format(n_iter, accuracy, precision, recall))
    
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print("\n")
#print("\n 평균 검증 정확도", np.mean(cv_accuracy))
print("\n 평균 검증 정확도", np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))


1 교차검증 정확도 :0.98, 교차검증 정밀도 :0.981 교차검증 재현율 :0.979

2 교차검증 정확도 :0.92, 교차검증 정밀도 :0.919 교차검증 재현율 :0.919

3 교차검증 정확도 :0.96, 교차검증 정밀도 :0.961 교차검증 재현율 :0.961



 평균 검증 정확도 0.9533333333333333 0.9536666666666666 0.953


Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

- 정답이 2개만 있으면 문제없이 계산할 수 있었는데, 정답이 3개라서 정답 계산하는 식을 바꿔야 한다.
- 그 정답식을 바꿔주는 인자가 average, micro, macro, weighted
- micro: 전체의 평균
- macro: 평균중의 평균
- weighted: 가중 평균

- Kfold를 사용했을 때 불균형에 대한 문제점을 꼭 인지하고, 코드를 작성할 때 진행해주셔야 한다.

### 필수과제1. kfold, skf를 사용시 average가 총 3개가 나왔다.
- micro
- macro
- weighted

- 이 세 개의 차이점을 개념 정리해주시고, 직접 코드로 작성해서 정확도, 정밀도, 재현율이 어떻게 달라지는지 보여주세요!
- iris 데이터를 가지고 진행

### 필수과제2
- 우리가 모델을 평가하는 지표가 있다.
- 정확도, 정밀도, 재현율, f1스코어, roc-curve 등이 있다.
- 평가하는 지표가 어떤 식으로 구성되어있는지 개념만 정리해서 제출해주세요!
- 코드를 작성할 필요는 없다.

### 필수과제3
- sales 데이터를 드릴 예정
- 해당 sales 데이터 중에서 가장 많이 판매되었던 상품을 1, 2 3, 4까지 라벨링해주시고 해당 라벨링 값을 정답으로 하여
- 위에 배운 내용을 복습하는 코드를 작성해주세요!
- 해당 데이터는 정답값이 없다. 우리가 정답값을 만들어야한다.

- 그 정답값을 만들어서 DecisionTree 모델만 사용하여 진행
- skf를 이용해서 동일하게 micro, macro, weighted를 비교하여 정리해주시고
- 데이터의 전처리의 경우 시간이 오래 걸릴 수 있으니 최대한 시간 효율 보셔서 진행해주시면 된다.
- 다만 데이터 전처리가 없으면 안 됩니다.
- 1, 2, 3, 4 외에 나머지 데이터는 0로 해주시고 데이터 전체 삭제하지 마시고 분류해주세요.
- 1, 2, 3, 4, 0 라벨값이 생성될 것

- 과제에 대한 지정자를 선정하여 네이버 카페에 올려주셔야 합니다.
- 자발적으로 제출하고 싶으신 경우는 방법도 알려드립니다.

- 팀별과제는 곧 정리하셔서 영상촬영 예정이며, 전처리에 대한 내용을 5~7분 코드 설명하는 영상으로 정리하셔야 합니다.